In [39]:
import nltk
from nltk import bigrams,trigrams 
from nltk.corpus import reuters
from collections import Counter, defaultdict
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus as wkc


In [40]:
wikidspath = datapath('enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2')
sentences = wkc(wikidspath).get_texts()

In [54]:
nltk.download('punkt')
nltk.download('reuters')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [41]:
rs  = reuters.sents()

In [42]:
print(rs)

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


In [47]:
def wordsProbability(sentenceModel):
  for nextWord in sentenceModel:
    nextWords = sentenceModel[nextWord]
    totalWordCount = float(sum(nextWords.values()))
    for previousWord in nextWords:
      nextWords[previousWord] /= totalWordCount

def singleWordProbability(sentenceModel,wordCount):
  for word in sentenceModel:
    sentenceModel[word] /= wordCount

def convertToLower(s):
  if type(s)==str:
    return s.lower()
  return s

In [29]:
sentenceModel4 = defaultdict(lambda: set()) # default value of model's keys is set as set
sentenceModel5 = defaultdict(lambda: set())

# calculating overall word count in the given sentence model
def calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,sentences):
  wordCount = 0
  for sentence in sentences:
    for word in sentence:
      wordCount += 1
      sentenceModel1[word] += 1 #storing counts of each word in the first model
    for previousWord2,previousWord1,nextWord in trigrams(sentence,pad_right=True,pad_left=True):
      previousWord1 = convertToLower(previousWord1)
      previousWord2 = convertToLower(previousWord2)
      nextWord = convertToLower(nextWord)
      sentenceModel2[nextWord][previousWord1] += 1 # storing count of just the previous words in case of specific word occuring
      sentenceModel3[nextWord][previousWord2] += 1 # storing count of 2nd previous words in case of specific word occuring
      sentenceModel4[previousWord1].add(nextWord) # adding new word based on the just previous word in the trigram to the sentence model
      sentenceModel5[previousWord2].add(nextWord) # adding new word based on the 2nd previous word in the trigram to the sentence model

  return wordCount

In [30]:
sentenceModel1 = defaultdict(lambda:0)
sentenceModel2 = defaultdict(lambda: defaultdict(lambda:0)) # The argument will be called when we try to access a key that doesn't exist
sentenceModel3 = defaultdict(lambda: defaultdict(lambda:0))

WikiWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,WikiSentences) # counting words in wiki corpus dataset
print(WikiWordCount)

452944


In [21]:
ReutersWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,ReutersSentences) # counting words in Reuters dataset
print(ReutersWordCount)

1720917


In [31]:
calculateWordsProbabilities(sentenceModel2) # checking probabilities in model 2
calculateWordsProbabilities(sentenceModel3) # checking probabilities in model 3

totalWord = WikiWordCount + ReutersWordCount
calculateSingleWordProbability(sentenceModel1,totalWord) # checking each word probability in model 1

In [32]:
ProbabilityWordsList = [] 
# getting word suggestion by placing two words
def WordSuggestionsByTrigram(previousWord2,previousWord1): 
  for nextWord in sentenceModel4[previousWord1] & sentenceModel5[previousWord2]:
    naiveBayesTrigramValue = sentenceModel1[nextWord]*sentenceModel2[nextWord][previousWord1]*sentenceModel3[nextWord][previousWord2] # using naive bayes to get the weight of the each trigram 
    ProbabilityWordsList.append((nextWord,naiveBayesTrigramValue)) # storing the predicted words and the weights of trigram

In [33]:
# test of suggestions given by placing words "i have" 
WordSuggestionsByTrigram('i','have')
ProbabilityWordsList.sort(key=lambda o:o[1],reverse=True)
print(*ProbabilityWordsList[:10])

In [34]:
# test of suggestions given by placing words "i have" 
WordSuggestionsByTrigram('i','have')
ProbabilityWordsList.sort(key=lambda o:o[1],reverse=True)
print(*ProbabilityWordsList[:10])

In [53]:
text = input("Enter your Sentence: ")
ProbabilityWordsList = [] 
text = text.split(" ")
WordSuggestionsByTrigram(text[0],text[1])
ProbabilityWordsList.sort(key=lambda o:o[1],reverse=True)
print(*ProbabilityWordsList[:10])

Enter your Sentence: how to
('reintegrate', 4.600110126636432e-07) ('quell', 4.600110126636432e-07) ('repay', 4.600110126636432e-07) ('implement', 4.181918296942211e-07) ('determine', 3.9429515371169415e-07) ('interpret', 3.577863431828336e-07) ('achieve', 3.517731273310212e-07) ('build', 3.517731273310212e-07) ('identify', 2.587561946232993e-07) ('make', 2.425512612226482e-07)
